In [ ]:
import os
import json
import requests
import pandas as pd

In [ ]:
albums = pd.read_json("albums_0-999.json")
artists = pd.read_json("artists_0-999.json")
mpd = pd.read_json("mpd.slice.0-999.json")
songs = pd.read_json("songs_0-999.json")

In [ ]:
albums.head()

In [ ]:
artists.head()

In [ ]:
mpd.head()

In [ ]:
songs.head()